This work is to calculate the TF-IDF of words in documents provided in the bbctext folder

In [1]:
//libraries to import 

import scala.collection.immutable.WrappedString;
import scala.collection.Map

import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
//import sqlContext.implicits._            // import useful stuff
import org.apache.spark.sql.functions._  // here, import min, max, etc. 


In [2]:
//initialise Spark
val spark = SparkSession.builder.        // builder pattern to construct the session
  master("local[*]").                    // run locally on all cores ([*])
  appName("TF-IDF").
  config("spark.app.id", "Console").     // to silence Metrics warning
  getOrCreate()                          // create it!

val sc = spark.sparkContext              // get the SparkContext
val sqlContext = spark.sqlContext        // get the old SQLContext, if needed




spark = org.apache.spark.sql.SparkSession@1bbfbc89
sc = org.apache.spark.SparkContext@39ab7567
sqlContext = org.apache.spark.sql.SQLContext@5c1e212b


org.apache.spark.sql.SQLContext@5c1e212b

In [3]:
spark.getClass

class org.apache.spark.sql.SparkSession

In [4]:
sc.getClass

class org.apache.spark.SparkContext

In [5]:
//initialise the variable to hold the business folder
val bus_text = sc.wholeTextFiles("../TF-IDF/bbctext/business")

bus_text = ../TF-IDF/bbctext/business MapPartitionsRDD[1] at wholeTextFiles at <console>:35


../TF-IDF/bbctext/business MapPartitionsRDD[1] at wholeTextFiles at <console>:35

In [13]:
//initialise the variable to hold the tech folder
val tech_text = sc.wholeTextFiles("../TF-IDF/bbctext/tech")

tech_text = ../TF-IDF/bbctext/tech MapPartitionsRDD[17] at wholeTextFiles at <console>:35


../TF-IDF/bbctext/tech MapPartitionsRDD[17] at wholeTextFiles at <console>:35

In [14]:
//initialise the variable to hold the sports folder
val sport_text = sc.wholeTextFiles("../TF-IDF/bbctext/sport")

sport_text = ../TF-IDF/bbctext/sport MapPartitionsRDD[19] at wholeTextFiles at <console>:35


../TF-IDF/bbctext/sport MapPartitionsRDD[19] at wholeTextFiles at <console>:35

In [15]:
//initialise the variable to hold the politics folder
val pol_text = sc.wholeTextFiles("../TF-IDF/bbctext/politics")

pol_text = ../TF-IDF/bbctext/politics MapPartitionsRDD[21] at wholeTextFiles at <console>:35


../TF-IDF/bbctext/politics MapPartitionsRDD[21] at wholeTextFiles at <console>:35

In [16]:
//initialise the variable to hold the business folder
val ent_text = sc.wholeTextFiles("../TF-IDF/bbctext/entertainment")

ent_text = ../TF-IDF/bbctext/entertainment MapPartitionsRDD[23] at wholeTextFiles at <console>:35


../TF-IDF/bbctext/entertainment MapPartitionsRDD[23] at wholeTextFiles at <console>:35

In [7]:
//get size of business folder
bus_text.count() 

510

In [18]:
//get size of business folder
sport_text.count() 

511

In [19]:
//get size of business folder
ent_text.count() 

386

In [20]:
//get size of business folder
pol_text.count() 

417

In [21]:
//show the content of each file
bus_text.take(5).foreach(println)  

(file:/Users/drcaptainray/Documents/IMTA/SDS202/TP/TF-IDF/bbctext/business/289.txt,UK economy facing 'major risks'

The UK manufacturing sector will continue to face "serious challenges" over the next two years, the British Chamber of Commerce (BCC) has said.

The group's quarterly survey of companies found exports had picked up in the last three months of 2004 to their best levels in eight years. The rise came despite exchange rates being cited as a major concern. However, the BCC found the whole UK economy still faced "major risks" and warned that growth is set to slow. It recently forecast economic growth will slow from more than 3% in 2004 to a little below 2.5% in both 2005 and 2006.

Manufacturers' domestic sales growth fell back slightly in the quarter, the survey of 5,196 firms found. Employment in manufacturing also fell and job expectations were at their lowest level for a year.

"Despite some positive news for the export sector, there are worrying signs for manufacturing," t

In [29]:
//get stop words
val stopwords = sc.textFile("../TF-IDF/bbctext/stopwords.txt")

stopwords = ../TF-IDF/bbctext/stopwords.txt MapPartitionsRDD[32] at textFile at <console>:35


../TF-IDF/bbctext/stopwords.txt MapPartitionsRDD[32] at textFile at <console>:35

In [30]:
val stopwords_size : Int = stopwords.count().asInstanceOf[Int]

stopwords_size = 429


429

In [31]:
//read the content of stop words
stopwords.take(stopwords_size).foreach(println)

a
about
above
across
after
again
against
all
almost
alone
along
already
also
although
always
among
an
and
another
any
anybody
anyone
anything
anywhere
are
area
areas
around
as
ask
asked
asking
asks
at
away
b
back
backed
backing
backs
be
became
because
become
becomes
been
before
began
behind
being
beings
best
better
between
big
both
but
by
c
came
can
cannot
case
cases
certain
certainly
clear
clearly
come
could
d
did
differ
different
differently
do
does
done
down
down
downed
downing
downs
during
e
each
early
either
end
ended
ending
ends
enough
even
evenly
ever
every
everybody
everyone
everything
everywhere
f
face
faces
fact
facts
far
felt
few
find
finds
first
for
four
from
full
fully
further
furthered
furthering
furthers
g
gave
general
generally
get
gets
give
given
gives
go
going
good
goods
got
great
greater
greatest
group
grouped
grouping
groups
h
had
has
have
having
he
her
here
herself
high
high
high
higher
highest
him
himself
his
how
however
i
if
important
in
interest
interested
inter

In [32]:
val s_words = stopwords.flatMap(line => line.split(","))

s_words = MapPartitionsRDD[33] at flatMap at <console>:34


MapPartitionsRDD[33] at flatMap at <console>:34

In [33]:
val s_wordGroups = s_words.groupBy(word => word)

s_wordGroups = ShuffledRDD[35] at groupBy at <console>:34


ShuffledRDD[35] at groupBy at <console>:34

In [12]:
import org.apache.spark.rdd.RDD

def peek(rdd: RDD[_],n: Int = 20): Unit = {
  println("RDD type signature: "+rdd+"\n")
  println("=====================")
  rdd.take(n).foreach(println)
  println("=====================")
}

peek: (rdd: org.apache.spark.rdd.RDD[_], n: Int)Unit


In [35]:
peek(s_wordGroups,stopwords_size)

RDD type signature: ShuffledRDD[35] at groupBy at <console>:34

(someone,CompactBuffer(someone))
(under,CompactBuffer(under))
(its,CompactBuffer(its))
(greater,CompactBuffer(greater))
(opening,CompactBuffer(opening))
(have,CompactBuffer(have))
(needing,CompactBuffer(needing))
(highest,CompactBuffer(highest))
(order,CompactBuffer(order))
(several,CompactBuffer(several))
(behind,CompactBuffer(behind))
(we,CompactBuffer(we))
(been,CompactBuffer(been))
(thing,CompactBuffer(thing))
(who,CompactBuffer(who))
(grouped,CompactBuffer(grouped))
(however,CompactBuffer(however))
(over,CompactBuffer(over))
(ever,CompactBuffer(ever))
(seeming,CompactBuffer(seeming))
(any,CompactBuffer(any))
(largely,CompactBuffer(largely))
(make,CompactBuffer(make))
(problems,CompactBuffer(problems))
(years,CompactBuffer(years))
(kind,CompactBuffer(kind))
(knows,CompactBuffer(knows))
(out,CompactBuffer(out))
(presented,CompactBuffer(presented))
(me,CompactBuffer(me))
(d,CompactBuffer(d))
(parts,CompactBuffer(parts))


In [37]:
val wordCounts = s_wordGroups.mapValues(group => group.size)
peek(wordCounts,stopwords_size)

RDD type signature: MapPartitionsRDD[36] at mapValues at <console>:39

(someone,1)
(under,1)
(its,1)
(greater,1)
(opening,1)
(have,1)
(needing,1)
(highest,1)
(order,1)
(several,1)
(behind,1)
(we,1)
(been,1)
(thing,1)
(who,1)
(grouped,1)
(however,1)
(over,1)
(ever,1)
(seeming,1)
(any,1)
(largely,1)
(make,1)
(problems,1)
(years,1)
(kind,1)
(knows,1)
(out,1)
(presented,1)
(me,1)
(d,1)
(parts,1)
(are,1)
(p,1)
(ended,1)
(away,1)
(presents,1)
(when,1)
(ends,1)
(our,1)
(as,1)
(whose,1)
(going,1)
(together,1)
(thus,1)
(v,1)
(better,1)
(them,1)
(myself,1)
(parted,1)
(anybody,1)
(showed,1)
(sides,1)
(backed,1)
(is,1)
(long,1)
(newest,1)
(same,1)
(even,1)
(backs,1)
(b,1)
(rather,1)
(certain,1)
(shall,1)
(come,1)
(n,1)
(shows,1)
(j,1)
(they,1)
(new,2)
(again,1)
(likely,1)
(goods,1)
(gives,1)
(my,1)
(now,1)
(has,1)
(four,1)
(us,1)
(z,1)
(men,1)
(after,1)
(areas,1)
(puts,1)
(need,1)
(turns,1)
(uses,1)
(places,1)
(anyone,1)
(ways,1)
(point,1)
(fact,1)
(three,1)
(yours,1)
(why,1)
(mrs,1)
(often,1)
(qu

wordCounts = MapPartitionsRDD[36] at mapValues at <console>:39


MapPartitionsRDD[36] at mapValues at <console>:39

In [47]:
def tokenize(line:String) : Array[String] = new WrappedString(line).toLowerCase().split("\\W+")

tokenize: (line: String)Array[String]


In [6]:
val bus_text_map = bus_text.map(line => line.toLowerCase)

bus_text_map = MapPartitionsRDD[2] at map at <console>:37


MapPartitionsRDD[2] at map at <console>:37

In [8]:
val new_tokenizedtext = bus_text.flatMap(line => line.split("""[^\p{IsAlphabetic}]+"""))

new_tokenizedtext = MapPartitionsRDD[4] at flatMap at <console>:37


MapPartitionsRDD[4] at flatMap at <console>:37

In [13]:
peek(new_tokenizedtext)

RDD type signature: MapPartitionsRDD[4] at flatMap at <console>:37

UK
economy
facing
major
risks

The
UK
manufacturing
sector
will
continue
to
face
serious
challenges
over
the
next
two


In [14]:
val bustext_wordGroups = bus_text_map .groupBy(word => word)
peek(bustext_wordGroups)

RDD type signature: ShuffledRDD[6] at groupBy at <console>:41

("emi music's sales, particularly re-orders, in january have also been lower than anticipated and this is expected to continue through february and march," the company added. "therefore, for the full year, at constant currency, emi music's sales are now expected to be 8% to 9% lower than the prior year." the company said it expected profits to be about £138m ($259.8m). alain levy, chairman and chief executive of emi music, described the performance as "disappointing", but added that he remained optimistic over future trends in the industry. "the physical music market is showing signs of stabilisation in many parts of the world and digital music, in all its forms, continues to develop at a rapid pace," he said.,CompactBuffer("emi music's sales, particularly re-orders, in january have also been lower than anticipated and this is expected to continue through february and march," the company added. "therefore, for the full year

bustext_wordGroups = ShuffledRDD[6] at groupBy at <console>:41


ShuffledRDD[6] at groupBy at <console>:41

In [15]:
val wordCounts1 = bustext_wordGroups.map( word_group => (word_group._1, word_group._2.size))
peek(wordCounts1)

RDD type signature: MapPartitionsRDD[7] at map at <console>:41

("emi music's sales, particularly re-orders, in january have also been lower than anticipated and this is expected to continue through february and march," the company added. "therefore, for the full year, at constant currency, emi music's sales are now expected to be 8% to 9% lower than the prior year." the company said it expected profits to be about £138m ($259.8m). alain levy, chairman and chief executive of emi music, described the performance as "disappointing", but added that he remained optimistic over future trends in the industry. "the physical music market is showing signs of stabilisation in many parts of the world and digital music, in all its forms, continues to develop at a rapid pace," he said.,1)
(,2172)
(ba to suspend two saudi services,1)
(new york attorney general elliot spitzer said senior vice president robert stearns had pleaded guilty to scheming to defraud. the offence carries a sentence of 16 mont

wordCounts1 = MapPartitionsRDD[7] at map at <console>:41


MapPartitionsRDD[7] at map at <console>:41

In [45]:
val tokenized_text = tech_text.map(t => (t._1.toString,tokenize(t._2.toString)))

lastException = null


Name: org.apache.spark.SparkException
Message: Task not serializable
StackTrace:   at org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:403)
  at org.apache.spark.util.ClosureCleaner$.org$apache$spark$util$ClosureCleaner$$clean(ClosureCleaner.scala:393)
  at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:162)
  at org.apache.spark.SparkContext.clean(SparkContext.scala:2326)
  at org.apache.spark.rdd.RDD$$anonfun$map$1.apply(RDD.scala:371)
  at org.apache.spark.rdd.RDD$$anonfun$map$1.apply(RDD.scala:370)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
  at org.apache.spark.rdd.RDD.map(RDD.scala:370)
  ... 46 elided
Caused by: java.io.NotSerializableException: org.apache.spark.SparkContext
Serialization stack:
	- object not serializable (class: org.apache.spark.SparkCont

In [112]:
val term_frequency = tokenized_text.flatMapValues(x => x).countByValue()

Name: Unknown Error
Message: lastException: Throwable = null
<console>:63: error: not found: value tokenized_text
       val term_frequency = tokenized_text.flatMapValues(x => x).countByValue()
                            ^

StackTrace: 